# Find feature = furnished

In [36]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.
This is by no means production code.
"""
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup, SoupStrainer
from urllib.request import urlopen
import requests

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 50) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?features=furnished&sort=price-desc&page={page}"
    bs_object = BeautifulSoup(requests.get(url,headers = headers).text, "html.parser")
    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    if bs_object is not None:
        index_links = bs_object \
            .find(
                "ul",
                {"data-testid": "results"}
            ) \
            .findAll(
                "a",
                href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
            )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])

In [ ]:
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    bs_object = BeautifulSoup(requests.get(property_url,headers = headers).text, "html.parser")

    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]
    
    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    
    
    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/

with open('/Users/tianhao/ads_asmt2/furnished.json', 'w') as f:
    dump(property_metadata, f)

# find feature = unfurnished

In [26]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.
This is by no means production code.
"""
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup, SoupStrainer
from urllib.request import urlopen
import requests

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 2) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?sort=price-desc&page={page}"
    bs_object = BeautifulSoup(requests.get(url,headers = headers).text, "html.parser")
    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    if bs_object is not None:
        index_links = bs_object \
            .find(
                "ul",
                {"data-testid": "results"}
            ) \
            .findAll(
                "a",
                href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
            )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])



In [27]:
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    bs_object = BeautifulSoup(requests.get(property_url,headers = headers).text, "html.parser")

    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]
    
    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    
    
    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/

with open('/Users/tianhao/ads_asmt2/unfiltered.json', 'w') as f:
    dump(property_metadata, f)

In [28]:
import pandas as pd
df_furnished = pd.read_json('furnished.json')
df_raw = pd.read_json('unfiltered.json')

In [29]:
df_furnished = df_furnished.T
df_raw = df_raw.T

In [30]:
df_furnished = df_furnished.reset_index(drop=True)
df_raw = df_raw.reset_index(drop=True)

In [32]:
df_raw['Furnished'] = ' '

In [34]:
for i in range(0,len(df_raw)):
    for j in range(0,len(df_furnished)):
        if df_furnished['name'][j] == df_raw['name'][i]:
            df_raw['Furnished'][i] = 'True'
        else:
            df_raw['Furnished'][i] = 'False'

In [35]:
df_raw

,name,cost_text,coordinates,rooms,desc,Furnished
0,667 Glenhuntly Road Caulfield VIC 3162,"$38,000 p.a. Incl. Outgoings + GST","[-37.8860233, 145.0173065]",[],Can you hear it? Opportunity knocking!!,False
1,Upstairs 2C Staley Street Brunswick VIC 3056,"$35,000 Annually","[-37.7655919, 144.9633048]","[[0 Beds], [0 Baths], [2 Parking]]",1st floor offices/studios\nEasy walking distan...,False
2,"Level 3, 302/13-15 Lake Street Caroline Spring...",4125000 pw,"[-37.7316459, 144.7446886]",[],- Centrally located with tranquil views.\n- Le...,False
3,3502/14-16 The Esplanade St Kilda VIC 3182,"$6,000","[-37.8650177, 144.9746821]","[[3 Beds], [3 Baths], [3 Parking]]",Inspired by the interplay of timeless design a...,False
4,9 Lansdowne Street Blairgowrie VIC 3942,"$5,000 per week","[-38.372703, 144.7856897]","[[3 Beds], [2 Baths], [3 Parking], []]",Phone enquiry code for this property : 2751,False
5,52 Black St Brighton VIC 3186,From $5500 per week,"[-37.9159452, 144.9989003]","[[3 Beds], [4 Baths], [3 Parking], []]","class=""css-dxogle"">* Unverified feature<svg a...",False
6,4203/35 Spring Street Melbourne VIC 3000,$4500 Per Week,"[-37.8141725, 144.9740049]","[[3 Beds], [2 Baths], [4 Parking]]",Possibly the most prestigious CBD address in t...,False
7,440 Keys Road Flinders VIC 3929,"$4,000.00","[-38.4788173, 144.9643428]","[[6 Beds], [4 Baths], [2 Parking]]",Boasting sublime architectural design and beau...,False
8,335 The Esplanade Indented Head VIC 3223,"$3,775 wk $545 pn","[-38.1439923, 144.7147899]","[[4 Beds], [3 Baths], []]",This stylish 4 bedroom home will surely make y...,False
9,50 South Wharf Drive Docklands VIC 3008,$3750 per week,"[-37.8223967, 144.938237]","[[5 Beds], [6 Baths], [3 Parking]]",This spectacular five bedroom freehold waterfr...,False
